In [1]:
# Install system packages needed for building C++ code
!apt-get update
!apt-get install -y cmake g++ build-essential git

# Clean previous clones (optional)
!rm -rf /content/dismecpp

# 1. Clone DiSMEC++ with all submodules
!git clone --recursive https://github.com/xmc-aalto/dismecpp.git /content/dismecpp

# 2. Fix pybind11 to compatible version
%cd /content/dismecpp/deps/pybind11
!git checkout v2.11.1

# 3. Create build directory and compile
%cd /content/dismecpp
!mkdir -p build
%cd build
!cmake .. && make -j4

# 4. Verify that the 'train' binary exists
!ls -l ./bin/train


Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,851 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,765 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-s

In [5]:
!git clone https://github.com/kunaldahiya/pyxclib.git
%cd pyxclib
!python3 setup.py install --user

In [ ]:
!ls

In [1]:
from xclib.data import data_utils

data_utils.save_data("path/to/amazoncat13k", "train", X_train, Y_train)
data_utils.save_data("path/to/amazoncat13k", "test", X_test, Y_test)

In [ ]:
from xclib.data import data_utils

# Read file with features and labels (old format from XMLRepo)
features, tabels, num_samples, num_features, num_labels = data_utils.read_data('train.txt')

# Read sparse file (see docstring for more)
# header can be set to false (if required)
labels = data_utils.read_sparse_file('trn_X_Xf.txt', header=True)

# Write sparse file (with header)
data_utils.write_sparse_file(labels, "labels.txt")

In [3]:
./build/dismec-train \
  --input path/to/amazoncat13k/train.txt \
  --model-dir path/to/models/amazoncat13k/ \
  --threads 8 \
  --max-learners 500 \
  --k 300 \
  --dual \
  --bias \
  --eps 0.001 \
  --verbose

In [2]:
./build/dismec-predict \
  --input path/to/amazoncat13k/test.txt \
  --model-dir path/to/models/amazoncat13k/ \
  --output path/to/predictions/amazoncat13k.svm \
  --threads 8 \
  --scores


In [ ]:
cd tools/java
javac -cp . MultiLabelMetrics.java
java -cp . MultiLabelMetrics \
  path/to/amazoncat13k/test.txt \
  path/to/predictions/amazoncat13k.svm

In [4]:
from xclib.eval import xc_metrics
true_labels = data_utils.read_sparse_file("path/to/amazoncat13k/test.txt")
pred_labels = data_utils.read_sparse_file("path/to/predictions/amazoncat13k.svm")

for k in [1, 3, 5]:
    print(f"P@{k}: ", xc_metrics.precision_at_k(true_labels, pred_labels, k))


from xclib.data import data_utils
import xclib.evaluation.xc_metrics as xc_metrics

# Read ground truth and predictions
true_labels = data_utils.read_sparse_file('tst_X_Y.txt')
predicted_labels = data_utils.read_sparse_file('parabel_predictions.txt')

# evaluate (See examples/evaluate.py for more details)
acc = xc_metrics.Metrics(true_labels=true_labels)
args = acc.eval(predicted_labels, 5)
print(xc_metrics.format(*args))